In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,DepthwiseConv2D, Flatten, Dense, BatchNormalization, Dropout, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.metrics import Precision, Recall, AUC
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

#### Dataset Load and Merge

In [1]:
import os
import cv2
import numpy as np

dataset_dir = "D:/xtras/CNN/Brain Tumor/Rebuttal/Alzheimer_s Dataset"

categories = ['VeryMildDemented', 'MildDemented', 'ModerateDemented', 'NonDemented']

mapped_dict = {'VeryMildDemented': 0, 'MildDemented': 0, 'ModerateDemented': 1, 'NonDemented': 2}

ad_data = []
labels = []

def process_images(category, split):
    path = os.path.join(dataset_dir, split, category)
    class_num = mapped_dict[category]
    # print(path.replace("/", "\\"))
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
            img_array = cv2.resize(img_array, (224, 224))
            ad_data.append([img_array,class_num])
        except Exception as e:
            pass

for category in categories:
    process_images(category, 'train')
    process_images(category, 'test')
print("Data shape:", len(ad_data))


Data shape: 6400


In [5]:
dataset_dir = "D:/xtras/CNN/Brain Tumor/Rebuttal/Axial"

categories = ['AD', 'CI', 'CN']
mapped_dict_2 = {'CI': 0, 'AD': 1, 'CN': 2}

adni_data = []
labels = []

for category in categories:
    path = os.path.join(dataset_dir, '', category)
    class_num = mapped_dict_2[category]

    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
        img_array = cv2.resize(img_array, (224, 224))
        adni_data.append([img_array, class_num])


In [6]:
len(adni_data)

5154

In [7]:
data = adni_data + ad_data
len(data)
np.random.shuffle(data)

#### Saving Folds to drive

In [3]:
from sklearn.model_selection import StratifiedKFold

In [11]:

X = []
y = []

for features, label in data:
    X.append(features)
    y.append(label)

X = np.array(X)
y = np.array(y)

# Initialize StratifiedKFold with 10 splits
skf = StratifiedKFold(n_splits=10)

base_path = "Alzheimer Merged Dataset"


for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    count = 0
    fold_dir = os.path.join(base_path, f"fold_{fold+1}")
    os.makedirs(fold_dir, exist_ok=True)

    train_dir = os.path.join(fold_dir, 'train')
    test_dir = os.path.join(fold_dir, 'test')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Get train and test splits
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Save training images
    for image_data, label in zip(X_train, y_train):
        class_dir = os.path.join(train_dir, f'class_{label}')
        os.makedirs(class_dir, exist_ok=True)
        count += 1
        save_path = os.path.join(class_dir, f"{count}.jpg")
        cv2.imwrite(save_path, cv2.cvtColor(image_data, cv2.COLOR_RGB2BGR))

    # Save test images
    for image_data, label in zip(X_test, y_test):
        class_dir = os.path.join(test_dir, f'class_{label}')
        os.makedirs(class_dir, exist_ok=True)
        count += 1
        save_path = os.path.join(class_dir, f"{count}.jpg")
        cv2.imwrite(save_path, cv2.cvtColor(image_data, cv2.COLOR_RGB2BGR))

    print(f"Fold {fold + 1} saved.")


Fold 1 saved.
Fold 2 saved.
Fold 3 saved.
Fold 4 saved.
Fold 5 saved.
Fold 6 saved.
Fold 7 saved.
Fold 8 saved.
Fold 9 saved.
Fold 10 saved.
